In [ ]:
%run params.ipynb
import rules.example

for name, rules in [('example', rules.example.build_rules())]:
    PARAMS['save_file'] = True
    PARAMS['name'] = name
    PARAMS['graph'] = rules['graph']          
    PARAMS['rules'] = rules['rules']        
    
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [6]:
set([1,2,3]) - set([2])

{1, 3}

In [12]:
import re
import rules.v01_oscillating
from rules.enzymes import *

all_nodes = {Actin_BR, Actin_ST, Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, 
             Dia1, Dia2, Ena_Vasp, IRSp53, LIMK, PAK, PIP2, PKD, Profilin, 
             ROCK, ROCK_, Rac1, RhoA, SSH, Thymosin, WASP, WAVE}

rules_map = {}
orig_rules = rules.v01_oscillating.build_rules() 
# rules_map["v01_oscillating"] = orig_rules

# knockdown nodes
for node in all_nodes - {CP, Coronin, Arp2_3, Dia1, Dia2, Rac1, WAVE, Cortactin, Actin_BR, Actin_ST}:
    graph = orig_rules["graph"].copy()
    graph.remove_edges_from([(node, target) for target in list(graph[node])])      
    
    rules = re.sub('1:\s*'+node+'\*\s*=.+', '1: '+node+'* = False', orig_rules['rules'])    
    rules_map["v01_oscillating_del_"+node] = {"graph": graph, "rules": rules}

sorted(rules_map.keys())

['v01_oscillating_del_Cdc42',
 'v01_oscillating_del_Cofilin',
 'v01_oscillating_del_Ena_Vasp',
 'v01_oscillating_del_IRSp53',
 'v01_oscillating_del_LIMK',
 'v01_oscillating_del_PAK',
 'v01_oscillating_del_PIP2',
 'v01_oscillating_del_PKD',
 'v01_oscillating_del_Profilin',
 'v01_oscillating_del_ROCK',
 'v01_oscillating_del_ROCK_',
 'v01_oscillating_del_RhoA',
 'v01_oscillating_del_SSH',
 'v01_oscillating_del_Thymosin',
 'v01_oscillating_del_WASP']

In [13]:
### run selected rules

%run params.ipynb

for name, rules in sorted(rules_map.items()):
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 100
    PARAMS['chunks'] = 50
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST]]
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

====== v01_oscillating_del_Cdc42
Saving dependecny matrix for v01_oscillating_del_Cdc42
Running boolean simulations
Saved chunk 0
Saved chunk 1
Saved chunk 2
Saved chunk 3
Saved chunk 4
Saved chunk 5
Saved chunk 6
Saved chunk 7
Saved chunk 8
Saved chunk 9
Saved chunk 10
Saved chunk 11
Saved chunk 12
Saved chunk 13
Saved chunk 14
Saved chunk 15
Saved chunk 16
Saved chunk 17
Saved chunk 18
Saved chunk 19
Saved chunk 20
Saved chunk 21
Saved chunk 22
Saved chunk 23
Saved chunk 24
Saved chunk 25
Saved chunk 26
Saved chunk 27
Saved chunk 28
Saved chunk 29
Saved chunk 30
Saved chunk 31
Saved chunk 32
Saved chunk 33
Saved chunk 34
Saved chunk 35
Saved chunk 36
Saved chunk 37
Saved chunk 38
Saved chunk 39
Saved chunk 40
Saved chunk 41
Saved chunk 42
Saved chunk 43
Saved chunk 44
Saved chunk 45
Saved chunk 46
Saved chunk 47
Saved chunk 48
Saved chunk 49
Normalizing cycles
Reporting simple basins
Reporting special basins
Reporting weighted activity
====== v01_oscillating_del_Cofilin
Saving depend

ZeroDivisionError: division by zero

====== v01_oscillating_del_RhoA
Saving dependecny matrix for v01_oscillating_del_RhoA
Running boolean simulations
Saved chunk 0
Saved chunk 1
Saved chunk 2
Saved chunk 3
Saved chunk 4
Saved chunk 5
Saved chunk 6
Saved chunk 7
Saved chunk 8
Saved chunk 9
Saved chunk 10
Saved chunk 11
Saved chunk 12
Saved chunk 13
Saved chunk 14
Saved chunk 15
Saved chunk 16
Saved chunk 17
Saved chunk 18
Saved chunk 19
Saved chunk 20
Saved chunk 21
Saved chunk 22
Saved chunk 23
Saved chunk 24
Saved chunk 25
Saved chunk 26
Saved chunk 27
Saved chunk 28
Saved chunk 29
Saved chunk 30
Saved chunk 31
Saved chunk 32
Saved chunk 33
Saved chunk 34
Saved chunk 35
Saved chunk 36
Saved chunk 37
Saved chunk 38
Saved chunk 39
Saved chunk 40
Saved chunk 41
Saved chunk 42
Saved chunk 43
Saved chunk 44
Saved chunk 45
Saved chunk 46
Saved chunk 47
Saved chunk 48
Saved chunk 49
Normalizing cycles
Reporting simple basins
Reporting special basins
Reporting weighted activity
====== v01_oscillating_del_SSH
Saving dependecny m

In [ ]:
%run params.ipynb

arr = ['v01_oscillating', 'v01_oscillating_del_Arp2_3']

for name, rules in [(val, rules_map[val]) for val in arr]:
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 2
    PARAMS['chunks'] = 5
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST]]
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [ ]:
### run all default rules

import rule_factory

%run params.ipynb

for rules in sorted(rule_factory.loaders):
    PARAMS['save_file'] = True
    PARAMS['rules'] = rules      
    PARAMS['graph'] = rule_factory.get_rules(rules)["graph"]    
    
    %run "01. structural analysis.ipynb"